In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from IPython.display import HTML
import base64
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pickle


In [2]:
market_train_df = pickle.load(open('Market_train',"rb"))
news_train_df = pickle.load(open('News_train', 'rb'))

In [3]:
def preprocess_data(mkt_train_df, news_train_df):
    mkt_train_df['time'] = mkt_train_df['time'].dt.date
    news_train_df['time'] = news_train_df['time'].dt.date
    assetCodes = []
    index = 0
    for x in news_train_df['assetCodes']:
        x = x.split(',')[0].split("'")[1]
        assetCodes.append(x)
    news_train_df['assetCode'] = np.asarray(assetCodes)
    irrelevantColumns = ['sourceTimestamp', 'firstCreated', 'sourceId', 
                         'headline', 'provider', 'subjects', 'audiences',
                        'headlineTag', 'marketCommentary', 'assetCodes', 'assetName']
    news_train_df.drop(irrelevantColumns, axis=1, inplace=True)
    market_train_df.drop(['assetName'], axis=1, inplace=True)
    modifiednews = news_train_df.groupby(['time','assetCode'], sort=False).aggregate(np.mean).reset_index()
    
    # join news reports to market data, note many assets will have many days without news data
    merged = pd.merge(mkt_train_df, modifiednews, how='left', on=['time', 'assetCode'], copy=False) 
    merged = merged.fillna(0)
    return merged

In [4]:
def normalizeY(ydf):
    ydf = (ydf + 1) / 2
    return ydf

In [5]:
X = preprocess_data(market_train_df, news_train_df)
X = X[X['returnsOpenNextMktres10'] >=-1]
X = X[X['returnsOpenNextMktres10'] <=1]
y = X['returnsOpenNextMktres10']
y = normalizeY(y)
X.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)

assetCodesAndTime = X.iloc[:, :2]
X = X.iloc[:, 2:]

In [6]:
def regularize(df):
    for column in df:
        colmin = np.amin(df[column])
        colmax = np.amax(df[column])
        df[column] = (df[column] - colmin) / (colmax - colmin)
    return df

In [7]:
X = regularize(X)

In [30]:
def separateDfByStock(assetCodesAndTime, df):
    stockdict = {}
    for assetCode in assetCodesAndTime['assetCode'].unique():
        indices = assetCodesAndTime[assetCodesAndTime['assetCode'] == assetCode].index
        stockdict[assetCode] = df.iloc[indices]
    return stockdict

In [31]:
stockdict = separateDfByStock(assetCodesAndTime, X)

IndexError: positional indexers are out-of-bounds

4072203
